In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage

Syntetic Data Generation    

In [3]:
# Create a and initialize database with real estate listings
import random
from langchain_chroma import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_core.documents import Document
from uuid import uuid4

locations = [
    "San Francisco",
    "San Jose", 
    "Sunnyvale", 
    "Mountain View",
    "Palo Alto", 
    "Santa Clara", 
    "Cupertino", 
    "Milpitas"
    "Fremont", 
]

llm = ChatOpenAI(model="gpt-3.5-turbo")

embeddings = OpenAIEmbeddings()


vectorstore = Chroma(
    collection_name="example_collection",
    persist_directory="data",
    embedding_function=embeddings
)

parser = StrOutputParser()
print("Generating descriptions for the homes")
documents = []


for location in locations:
    bedrooms = random.randint(2, 5)
    bathrooms = random.randint(1, 3)
    house_size = random.randint(1000, 3000)
    price = random.randint(500000, 2000000)
    messages = [
        SystemMessage(f"""Generate a short description for the follwoing home. Please include detailed description of the home and surrounding area. The d
                      description should include information on the locality, transportation options, and nearby amenities. Please include 
                      details about the schools and parks in the area as well."""),
        HumanMessage(f"{bedrooms} bedroom, {bathrooms} bathroom house in {location} with {house_size} square feet for ${price}.")
    ]
    response = llm.invoke(messages)
    parser = StrOutputParser()
    doc = Document(page_content=parser.invoke(response))
    documents.append(doc)

    print(doc.page_content)
    print("\n==========================\n")
    

uuids = [str(uuid4()) for _ in range(len(documents))]
vectorstore.add_documents(documents=documents, ids=uuids)

/var/folders/wq/l4mf5wv16ll20mh0qjg52d9c0000gq/T/ipykernel_24035/1116868855.py:20: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo")
/var/folders/wq/l4mf5wv16ll20mh0qjg52d9c0000gq/T/ipykernel_24035/1116868855.py:22: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


Generating descriptions for the homes
Nestled in the vibrant city of San Francisco, this charming 5 bedroom, 1 bathroom house offers 1845 square feet of living space for $690,532. Located in a desirable neighborhood known for its diverse community and eclectic mix of shops and restaurants, this home provides a perfect blend of urban living and suburban tranquility.

Transportation options are abundant in the area, with easy access to public transportation including buses and trains, making commuting a breeze for residents. Nearby amenities include grocery stores, cafes, and local boutiques, providing convenience at every turn. 

For families with children, the neighborhood boasts top-rated schools within close proximity, ensuring quality education opportunities. Additionally, parks and recreational areas are scattered throughout the area, offering residents plenty of green spaces to enjoy outdoor activities and leisurely strolls.

Don't miss out on the chance to own this lovely home in

['c91601d9-66f6-419b-a3eb-3528ada57f8c',
 '84c7e9a9-103f-45f1-b393-0d9f1745cddd',
 '362edc7c-8abd-4aae-abda-33acd8e129d9',
 'd4217fe1-16e5-44b9-8adf-237e23a0eb01',
 'aa1f62d6-a5fb-493e-b840-04d749aba889',
 '0623e44d-1535-4128-9d2a-d777b7ad3019',
 'cba50a02-923b-437a-9107-dcb0c48518f4',
 '0d22d6c3-eef7-4af1-9318-b863c7f595d6']

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
def create_prompt():
    """
    Returns a prompt instructed to produce a rephrased question based on the user's
    last question, but referencing previous messages (chat history).
    """
    system_instruction = """You are a personalized real estate assistant. \
    Use the response from the user provided to questions below to tailor your personalized answer for the user \
    Provide detailed description of the property based on the users preferences. \
    """

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_instruction),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")])
    return prompt

In [11]:
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain

history_aware_retriever = create_history_aware_retriever(
    llm,
    vectorstore.as_retriever(),
    create_prompt()
)

from langchain.chains import create_retrieval_chain

system_prompt = (
    "You are a personalized real estate assistant. "
    "Use the following input from the user chat history to provide a detailed description of the property"
    "answer concise."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [14]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []
chat_history.extend(
    [
        AIMessage(content='How big do you want your house to be?'),
        HumanMessage(content='A comfortable three-bedroom house with a spacious kitchen and a cozy living room.'),
    ]
)
chat_history.extend(
    [
        AIMessage(content='What are 3 most important things for you in choosing this property?'),
        HumanMessage(content='A quiet neighborhood, good local schools, and convenient shopping options.'),
    ]
)
chat_history.extend(
    [
        AIMessage(content='Which amenities would you like?'),
        HumanMessage(content='A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.'),
    ]
)

chat_history.extend(
    [
        AIMessage(content='Which transportation options are important to you?'),
        HumanMessage(content='A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.'),
    ]
)

chat_history.extend(
    [
        AIMessage(content='How urban do you want your neighborhood to be?'),
        HumanMessage(content='A balance between suburban tranquility and access to urban amenities like restaurants and theaters.'),
    ]
)

docs = rag_chain.invoke({'input': 'Which home should I buy?', 'chat_history': chat_history })


## For this project we are going to take the top response and responsd to the user

print(docs['answer'])






Based on your preferences for a comfortable three-bedroom house in a quiet neighborhood with good local schools and convenient shopping options, along with a backyard for gardening, a two-car garage, and a modern, energy-efficient heating system, I recommend considering the charming 3-bedroom, 1-bathroom home in Fremont. 

This property offers a cozy retreat with a spacious layout, ample natural light, and convenient access to nearby amenities such as grocery stores, restaurants, and shopping centers. Additionally, the neighborhood boasts top-rated schools, parks, and recreational areas, making it an ideal choice for families. 

Fremont provides a balance between suburban tranquility and access to urban amenities, allowing you to enjoy a peaceful environment while still having convenient options for shopping and dining. The easy access to major highways and public transportation also ensures convenient commuting and exploration of the Bay Area.
